### Preprocessing

In [31]:
import os
from xml.dom.minidom import parse
import numpy as np
import math
import pandas as pd

classes = ['helmet','head','person']

def convert_annot(size , box):
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dhvscode-terminal:/5b16861ff020e4f5c6860a9be16ea53d/2
    return [x, y, w, h]

def save_txt_file(img_jpg_file_name, size, img_box):
    save_file_name = 'datasets/SHW/labels/' + img_jpg_file_name + '.txt'
    
    #file_path = open(save_file_name, "a+")
    with open(save_file_name ,'a+') as file_path:
        for box in img_box:
            if box[0] in classes:
                cls_num = classes.index(box[0])

                new_box = convert_annot(size, box[1:])

                file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")
        file_path.flush()
        file_path.close()

        
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'
    #print(img_path)

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data
   
    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)
        
        img_jpg_file_name = img_xml_file + '.jpg'
        img_box.append([cls_name, x1, y1, x2, y2])
  

    # test_dataset_box_feature(img_jpg_file_name, img_box)
    save_txt_file(img_xml_file, [img_w, img_h], img_box)

from pathlib import Path
from shutil import copyfile

def copy_data(file_list, img_labels_root, imgs_source, mode):

    root_file = Path('datasets/SHW/images/'+  mode)
    if not root_file.exists():
        print(f"Path {root_file} does not exit")
        os.makedirs(root_file)

    root_file = Path('datasets/SHW/labels/' + mode)
    if not root_file.exists():
        print(f"Path {root_file} does not exit")
        os.makedirs(root_file)

    for file in file_list:               
        img_name = file.replace('.png', '')        
        img_src_file = imgs_source + '/' + img_name + '.png'        
        label_src_file = img_labels_root + '/' + img_name + '.txt'
        # Copy image
        DICT_DIR = 'datasets/SHW/images/'  + mode
        img_dict_file = DICT_DIR + '/' + img_name + '.png'

        copyfile(img_src_file, img_dict_file)

        # Copy label
        DICT_DIR = 'datasets/SHW/labels/' + mode
        img_dict_file = DICT_DIR + '/' + img_name + '.txt'
        copyfile(label_src_file, img_dict_file)

In [26]:
files = os.listdir('datasets/SHW/Annotations/')
for file in files:
    get_xml_data('datasets/SHW/Annotations/', file.split(".")[0])

In [29]:
from sklearn.model_selection import train_test_split

image_list = os.listdir('datasets/SHW/Images')
train_list, test_list = train_test_split(image_list, test_size=0.2, random_state=42)
val_list, test_list = train_test_split(test_list, test_size=0.5, random_state=42)
print('total =',len(image_list))
print('train :',len(train_list))
print('val   :',len(val_list))
print('test  :',len(test_list))

total = 5000
train : 4000
val   : 500
test  : 500


In [1]:
copy_data(train_list, 'datasets/SHW/labels', 'datasets/SHW/Images', "train")
copy_data(val_list,   'datasets/SHW/labels', 'datasets/SHW/Images', "val")
copy_data(test_list,  'datasets/SHW/labels', 'datasets/SHW/Images', "test")

NameError: name 'copy_data' is not defined

### combine helmet annots

In [1]:
######################

In [2]:
import os

In [26]:
main_dir = 'datasets/ppe/labels/val/'
files = [f'{main_dir}{i}' for i in os.listdir(main_dir)]

In [27]:
for file in files:
    with open(file, 'r') as f:
        labels = f.readlines()
    
    for i in range(len(labels)):
        if labels[i][0] in '345':
            labels[i] = f'2{labels[i][1:]}'
    
    with open(file, 'w') as wr:
        for lb in labels:
            wr.write(lb)

In [ ]:
# names:
#    0 : person
#    1 : vest

#    2 : blue helmet

#    3 : red helmet
#    4 : white helmet
#    5 : yellow helmet

In [24]:
with open('test.txt', 'w') as wr:
    for lb in labels:
        wr.write(lb)

### train

In [1]:
from ultralytics import YOLO

# Load a model
name = 'yolov10n'
# model = YOLO(f'{name}.yaml')  # build a new model from YAML
model = YOLO(f'{name}.pt')  # load a pretrained model (recommended for training)
# model = YOLO(f'{name}.yaml').load(f'{name}.pt')  # build from YAML and transfer weights

AttributeError: Can't get attribute 'SCDown' on <module 'ultralytics.nn.modules.block' from '/home/b.kurmetbek/miniconda3/envs/tumar/lib/python3.10/site-packages/ultralytics/nn/modules/block.py'>

In [30]:
# Train the model
# results = model.train(data='dataset.yaml', epochs=100, imgsz=320, patience=10, device=2)
# results = model.train(data='dataset_shw.yaml', epochs=2, imgsz=320, patience=10, device=2)

results = model.train(data='dataset_ppe.yaml', epochs=500, imgsz=640, patience=30, device=1)

New https://pypi.org/project/ultralytics/8.2.15 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.219 🚀 Python-3.10.13 torch-2.0.1 CUDA:1 (Tesla V100-SXM2-32GB, 31744MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=dataset_ppe.yaml, epochs=500, patience=30, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=1, workers=8, project=None, name=train15, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, s

train: Scanning /home/b.kurmetbek/projects/ppe_projcet/datasets/ppe/labels/train... 1064 images, 0 backgrounds, 0 corrupt: 100%|█████████

train: New cache created: /home/b.kurmetbek/projects/ppe_projcet/datasets/ppe/labels/train.cache



val: Scanning /home/b.kurmetbek/projects/ppe_projcet/datasets/ppe/labels/val... 266 images, 0 backgrounds, 0 corrupt: 100%|██████████| 26

val: New cache created: /home/b.kurmetbek/projects/ppe_projcet/datasets/ppe/labels/val.cache


...

    102/500      26.1G     0.8021     0.4422      1.058         97        640: 100%|██████████| 67/67 [00:56<00:00,  1.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:02<00:00,  3.22it/s]

                   all        266       2001      0.918      0.848      0.901      0.537
Stopping training early as no improvement observed in last 30 epochs. Best results observed at epoch 72, best model saved as best.pt.
To update EarlyStopping(patience=30) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



102 epochs completed in 1.839 hours.
Optimizer stripped from runs/detect/train15/weights/last.pt, 136.7MB
Optimizer stripped from runs/detect/train15/weights/best.pt, 136.7MB

Validating runs/detect/train15/weights/best.pt...
Ultralytics YOLOv8.0.219 🚀 Python-3.10.13 torch-2.0.1 CUDA:1 (Tesla V100-SXM2-32GB, 31744MiB)
Model summary (fused): 268 layers, 68126457 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:03<00:00,  2.31it/s]


                   all        266       2001      0.894      0.858      0.902       0.54
                person        266        837       0.89      0.875      0.914      0.554
                  vest        266        388      0.856      0.825      0.863      0.511
                helmet        266        776      0.937      0.875      0.929      0.554
Speed: 0.1ms preprocess, 6.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/train15


In [5]:
model.val(data='dataset_ppe.yaml')

Ultralytics YOLOv8.0.188 🚀 Python-3.9.17 torch-2.0.1 CUDA:2 (Tesla V100-SXM2-32GB, 31744MiB)
val: Scanning /home/b.kurmetbek/projects/ppe_projcet/datasets/SHW/labels/test... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<00:00, 
val: New cache created: /home/b.kurmetbek/projects/ppe_projcet/datasets/SHW/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:03<00:00,  8.90it/s]
                   all        500       2492      0.886      0.739      0.829        0.5
                helmet        500       1823      0.879      0.762       0.83      0.509
                  head        500        669      0.892      0.716      0.828      0.491
Speed: 0.0ms preprocess, 1.0ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/val3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ffc483b5220>
fitness: 0.5327938465044838
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.50871,     0.49115,     0.49993])
names: {0: 'helmet', 1: 'head', 2: 'person'}
plot: True
results_dict: {'metrics/precision(B)': 0.8857742024707005, 'metrics/recall(B)': 0.7391612570482783, 'metrics/mAP50(B)': 0.8285562872834525, 'metrics/mAP50-95(B)': 0.4999313530845984, 'fitness': 0.5327938465044838}
save_dir: PosixPath('runs/detect/val3')
speed: {'preprocess': 0.0333399772644043, 'inference': 1.0267338752746582, 'loss': 0.0013127326965332031, 'postprocess': 1.3213481903076172}

In [31]:
# !git clone https://github.com/ultralytics/yolov5

In [32]:
# !python yolov5/train.py --data dataset_shw.yaml --epochs 100 --cfg yolov5x.yaml  --batch-size 64 --imgsz 320 --patience 10 --device 2

In [33]:
detect_model = YOLO('runs/detect/train15/weights/best.pt')

In [39]:
result, = detect_model.predict('ppe_1320.jpg', imgsz=640, verbose=False, half=True, device=1)

In [46]:
import cv2
cv2.imwrite('output_image.jpg', result.plot())

True

In [47]:
# cap_path = '../red_zone/datasets/test_2.mp4'

# video_counter = 0
# counter = 0
# frame_num = 2
# flag = 0
# final_conf = 0.7
# n = 10

# info_list = {
#     'points': [], 
#     'wrists': [], 
#     'frames': []
# }
# detected = []
# outputs = []
# h, w = 1520, 2688

# cap = cv2.VideoCapture(cap_path)
# while cap.isOpened():
#     success, frame = cap.read()
#     if (not success) or (frame is None):
#         break
#     res, = detect_model.predict(frame, imgsz=640, verbose=False, half=True, device=1)
#     classes = res.boxes.cls.to().cpu().int().tolist()

# print('Detected cases:', counter)
# print('Frames:', frame_num)

In [ ]:
import cv2
import numpy as np

# Function to overlay an icon on an image
def overlay_icon(image, icon, position, size=(50, 50)):
    icon = cv2.resize(icon, size)
    x, y = position
    y1, y2 = y, y + icon.shape[0]
    x1, x2 = x, x + icon.shape[1]

    alpha_icon = icon[:, :, 3] / 255.0
    alpha_image = 1.0 - alpha_icon

    for c in range(0, 3):
        image[y1:y2, x1:x2, c] = (alpha_icon * icon[:, :, c] + 
                                  alpha_image * image[y1:y2, x1:x2, c])

    return image

# Load icons (ensure they have alpha channel for transparency)
helmet_icon = cv2.imread('helmet.png', cv2.IMREAD_UNCHANGED)
no_helmet_icon = cv2.imread('no_helmet.png', cv2.IMREAD_UNCHANGED)

# Dummy YOLO output (bounding boxes, class labels, and confidence scores)
# Replace this with your actual YOLO model output
detections = [
    {'bbox': (100, 100, 200, 200), 'label': 'person', 'has_helmet': True},
    {'bbox': (300, 100, 400, 200), 'label': 'person', 'has_helmet': False}
]

# Load the image
image = cv2.imread('ppe_1320.jpg')

# Process each detection
for detection in detections:
    x1, y1, x2, y2 = detection['bbox']
    
    if detection['label'] == 'person':
        # Determine which icon to use
        if detection['has_helmet']:
            icon = helmet_icon
        else:
            icon = no_helmet_icon
        # Overlay the icon on the image
        image = overlay_icon(image, icon, (x1, y1 - 50))  # Adjust position as needed

# Save or display the resulting image
cv2.imwrite('output_image.jpg', image)

# cv2.imshow('Output', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [59]:
# [x1, y1, x2, y2, conf, cls_name]

result.boxes.data.to().cpu().tolist()

[[257.421875, 55.859375, 298.046875, 195.1171875, 0.86279296875, 0.0],
 [193.1640625, 48.779296875, 238.0859375, 180.859375, 0.8583984375, 0.0],
 [39.74609375, 82.03125, 107.03125, 151.3671875, 0.83935546875, 0.0],
 [208.984375, 50.439453125, 233.984375, 68.017578125, 0.7958984375, 2.0],
 [193.1640625, 75.5859375, 234.1796875, 128.125, 0.7919921875, 1.0],
 [291.9921875, 30.46875, 343.5546875, 144.7265625, 0.779296875, 0.0],
 [272.65625, 56.640625, 293.75, 72.265625, 0.740234375, 2.0],
 [291.40625, 31.25, 309.375, 52.05078125, 0.72021484375, 2.0]]

In [54]:
cls_names = {
    0: 'person', 
    1: 'vest', 
    2: 'helmet'
}

{0: 'person', 1: 'vest', 2: 'helmet'}